In [239]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD, Adam
from tensorflow.keras import regularizers

In [240]:
# Read data
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
for_result_test_data = pd.read_csv("test.csv")

In [241]:
# Remove un-use columns

def drop_columns(df, list_columns=[]):
  return df.drop(list_columns, axis =1 )

list_columns =  ["PassengerId", "Name", "Ticket", "Fare", "Cabin", "Embarked"]
train_data = drop_columns(train_data, list_columns)
list_columns_test =  ["PassengerId", "Name", "Ticket", "Fare", "Cabin", "Embarked"]
test_data = drop_columns(test_data, list_columns_test)


In [242]:
# Remove nan in exist columns

train_data.fillna(train_data.mean(),inplace=True)
test_data.fillna(test_data.mean(),inplace=True)

In [243]:
# Split purpose

train_data_y = train_data.Survived
list_columns_survived =  ["Survived"]
train_data = drop_columns(train_data, list_columns_survived)

In [244]:
# one-hot vector

def one_hot_vector(df, list_OHV_columns):
  for column_name in list_OHV_columns:
    df = pd.get_dummies(df, columns=[column_name], prefix="oh"+column_name, sparse=True)
  return df

one_hot_columns = ["Pclass"]
train_data = one_hot_vector(train_data, one_hot_columns)
test_data = one_hot_vector(test_data, one_hot_columns)


In [245]:
# label encoding
sex_le = LabelEncoder()
train_data.Sex = sex_le.fit_transform(train_data.Sex)
test_data.Sex = sex_le.fit_transform(test_data.Sex)

In [246]:
# Standardization

sc = StandardScaler()
train_data.Age = sc.fit_transform(train_data.Age.values.reshape(-1, 1))
test_data.Age = sc.transform(test_data.Age.values.reshape(-1,1))

In [247]:
# to numpy
X = train_data.values
y = train_data_y.values

In [248]:
# Split data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [249]:
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [250]:
model = Sequential()
model.add(Dense(100, activation = "relu", input_dim = X_train.shape[1], name = "layer_1", kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(50, activation = "sigmoid", name = "layer_2", kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(25, activation = "sigmoid", name = "layer_3" , kernel_regularizer=regularizers.l2(0.001)))
model.add(Dense(1, activation = "sigmoid", name ="output_layer")) 
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['acc'])


In [251]:
model.fit(X_train, y_train, epochs=100, batch_size=32)


Epoch 1/100
23/23 [==============================] - 1s 3ms/step - loss: 0.7698 - acc: 0.6096
Epoch 2/100
23/23 [==============================] - 0s 3ms/step - loss: 0.7535 - acc: 0.6138
Epoch 3/100
23/23 [==============================] - 0s 3ms/step - loss: 0.7370 - acc: 0.6110
Epoch 4/100
23/23 [==============================] - 0s 3ms/step - loss: 0.7187 - acc: 0.6348
Epoch 5/100
23/23 [==============================] - 0s 3ms/step - loss: 0.7030 - acc: 0.6250
Epoch 6/100
23/23 [==============================] - 0s 3ms/step - loss: 0.6870 - acc: 0.6320
Epoch 7/100
23/23 [==============================] - 0s 3ms/step - loss: 0.6642 - acc: 0.6840
Epoch 8/100
23/23 [==============================] - 0s 3ms/step - loss: 0.6362 - acc: 0.7317
Epoch 9/100
23/23 [==============================] - 0s 3ms/step - loss: 0.6353 - acc: 0.7346
Epoch 10/100
23/23 [==============================] - 0s 3ms/step - loss: 0.6016 - acc: 0.7669
Epoch 11/100
23/23 [==============================] - 0s 2m

In [252]:

score = model.evaluate(X_train, y_train)
print("")
print("Train loss:{0}".format(score[0]))
print("Train accuracy:{0}".format(score[1]))
t_score = model.evaluate(X_test, y_test)
print("")
print("Test loss:{0}".format(t_score[0]))
print("Test accuracy:{0}".format(t_score[1]))



23/23 [==============================] - 0s 2ms/step - loss: 0.4720 - acc: 0.8244

Train loss:0.4719923734664917
Train accuracy:0.824438214302063
6/6 [==============================] - 0s 2ms/step - loss: 0.4548 - acc: 0.8101

Test loss:0.4548081159591675
Test accuracy:0.8100558519363403


In [253]:
result = model.predict(test_data.values)

14/14 [==============================] - 0s 2ms/step


In [254]:
df_suv = pd.DataFrame(result, columns = ["Survive"])

In [255]:
df_suv.Survive = round(df_suv.Survive)
df_suv.Survive = df_suv.Survive.astype('int8')

In [256]:
df_r = pd.concat([for_result_test_data, df_suv], axis=1)

In [257]:
df_r.drop(columns=[ "Name", "Ticket", "Fare", "Cabin", "Embarked",'Sex', 'Age', 'SibSp', 'Parch', 'Pclass'],inplace =True)

In [258]:
df_r

,PassengerId,Survive
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [264]:
df_r.to_csv("test_result", index = False)

In [260]:
df_r.columns

Index(['PassengerId', 'Survive'], dtype='object')

In [261]:
df_r

,PassengerId,Survive
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [262]:
df_suv

,Survive
0,0
1,0
2,0
3,0
4,1
...,...
413,0
414,1
415,0
416,0


In [263]:
test_data

,Sex,Age,SibSp,Parch,ohPclass_1,ohPclass_2,ohPclass_3
0,1,0.369449,0,0,0,0,1
1,0,1.331378,1,0,0,0,1
2,1,2.485693,0,0,0,1,0
3,1,-0.207709,0,0,0,0,1
4,0,-0.592481,1,1,0,0,1
...,...,...,...,...,...,...,...
413,1,0.044131,0,0,0,0,1
414,0,0.715743,0,0,1,0,0
415,1,0.677266,0,0,0,0,1
416,1,0.044131,0,0,0,0,1
